In [1]:
import numpy as np

In [ ]:
# Filter Configuration
D = 4.7 # cm
L = 25.4 # cm
A = np.pi*D**2/4
V = A*L # mL
VF = 2000 # mL/min

In [ ]:
def ode(t,y,k):
    Cs = y
    r = k*Cs
    dNsdt = -r
    dt = V/VF
    dCsdt = dNsdt/V-

    return [dCsdt]








In [ ]:
-0.3785ㅌ+4.8579 ( log의 PPT, x축 )